In [1]:
%run run.py 1

Selected Order: (3, 0, 0)
Using device: cpu
torch.Size([32, 10, 1])


KeyboardInterrupt: 

In [ ]:
%run real_data_exchange_linux.py 1

In [2]:
%run run.py 2

1.3663152077340701
